# Projet radiologie bootcamp DST : Etape 2, Preprocessing

### Plan de preprocessing : 
* Ouvrir chaque image avec son masque associé
* Convertir en Grayscale
* Redimensionner les images à la taille des masques (256 * 256)
* Appliquer les masques
* Equilibrer le nouveau dataset
* Séparer le dataset en ensemble d'entraînement (70%), de validation (15%) et de test (15%)

In [38]:
#Import des bibliothèques
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
import os, pathlib
from tqdm import tqdm
import cv2
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
import shutil #pour copier des fichiers vers les dossiers de preprocessing

### Etape 1 : création des images masquées et en échelles de gris
* Ouvrir chaque image en grayscale en itérant sur les dossiers
* Ouvrir le masque associé en grayscale
* Resizer l'image en 256*256
* Appliquer le masque
* Enregistrer l'image dans un nouveau dossier

In [16]:
#Sous-fonction qui ouvre une image à partir du filepath, la convertit en Grayscale et la redimensionne en 256*256
#En input : le filepath d'une image dans le dossier "image" => plutôt retravailler le masque
def load_and_resize(file):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (256, 256))
    return img_resized

#Sous-fonction qui va chercher le masque et le convertit en Grayscale
#En input : le filepath d'une image dans le dossier "image" qui est l'input de la fonction finale
def load_mask(file):
    mask_path = file.replace("images","masks")
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    return mask

#Fonction qui applique le masque et enregistre l'image ainsi préprocessée dans un nouveau dossier
#Input : image et masque (données d'origine), nom du fichier et chemin du dossier cible pour l'enregistrement
def get_masked_imgs(sourceimg, nom_cible, dossier_cible):
    img_resized = load_and_resize(sourceimg)
    mask = load_mask(sourceimg)
    img_masked = cv2.bitwise_and(img_resized, mask)
    cv2.imwrite(os.path.join(dossier_cible, nom_cible), img_masked)


In [19]:
#Test sur 1 fichier
fichier = "C:\\Users\\thiba\\Documents\\Projets data\\Projet-radio-DST_TG\\data\\Viral Pneumonia\\images\\Viral Pneumonia-14.png"
nom_cible = os.path.split(fichier)[1]
dossier_cible = "..\\data\\Preprocessing_1"

get_masked_imgs(fichier, nom_cible, dossier_cible)



#print(img_resized.shape)
#print(mask.shape)
#plt.figure(figsize=(15, 5))
#plt.subplot(131)
#plt.imshow(img_resized, cmap = "Greys_r")
#plt.subplot(132)
#plt.imshow(mask, cmap = "Greys_r")
#plt.subplot(133)
#plt.show()

In [21]:
#Définition des dossiers d'origine et cible, et lancement des fonctions => voir le script python dans SRC

#dossiers_source_dict = {"COVID" : "../data/COVID/images/", "NORMAL" : "../data/Normal/images/", "VIRAL_PNEUMONIA" : "../data/Viral Pneumonia/images/", "LUNG_OPACITY" : "../data/Lung_Opacity/images/" }
dossiers_source_dict = {"TEST" : "../data/test/images/"}
dossier_cible = "..\\data\\Preprocessing_1"

#Itérer sur les dossiers pour récupérer les fichiers images 
for n,filepath in dossiers_source_dict.items():
    for f in tqdm(os.listdir(filepath)):
        nom_cible = f
        file = os.path.join(filepath, f)
        get_masked_imgs(file, nom_cible, dossier_cible)




100%|██████████| 3/3 [00:00<00:00, 11.72it/s]


## Etape 2 : Equilibrage des classes par sous-échantillonage


In [22]:
#Import de la base excel metadata
meta_Covid = pd.read_excel("../data/COVID.metadata.xlsx")
meta_Covid.insert(loc = 0, column="CLASS", value =  "COVID")
meta_Normal = pd.read_excel("../data/Normal.metadata.xlsx")
meta_Normal.insert(loc = 0, column="CLASS", value =  "NORMAL")
meta_VPneu = pd.read_excel("../data/Viral Pneumonia.metadata.xlsx")
meta_VPneu.insert(loc = 0, column="CLASS", value =  "VIRAL_PNEUMONIA")
meta_LO = pd.read_excel("../data/Lung_Opacity.metadata.xlsx")
meta_LO.insert(loc = 0, column="CLASS", value =  "LUNG_OPACITY")
#Fusion des 4 metafichiers dans un dataframe
metadata = pd.concat([meta_Covid, meta_LO, meta_Normal, meta_VPneu], axis = 0)
#Enregistrement en csv
metadata.to_csv("../data/metadata_compil.csv")

In [32]:
#Nb de valeurs par classe dans metadata
print(len(metadata))
metadata["CLASS"].value_counts()

#Utilisation d'un RandomUnderSampler pour équilibrer le dataset
ru = RandomUnderSampler(random_state = 123, replacement = False)
meta_ru, y_ru = ru.fit_resample(metadata, metadata["CLASS"])
meta_ru.head()
meta_ru["CLASS"].value_counts()

21165


COVID              1345
LUNG_OPACITY       1345
NORMAL             1345
VIRAL_PNEUMONIA    1345
Name: CLASS, dtype: int64

In [42]:
meta_ru["CLASS"].value_counts().sum()

5380

In [50]:
## Sélection aléatoire et enregistrement des fichiers dans les dossiers TRAIN (70%), VALIDATE (15%), TEST (15%)
# Séparation jeu de TRAIN à 70/30
meta_TRAIN, meta_TESTVAL, y1, y2 = train_test_split(meta_ru, y_ru, test_size = 0.3, random_state = 123, stratify = y_ru)

#Séparation du jeu de VALIDATION et TEST à 50/50
meta_VAL, meta_TEST, y3, y4 = train_test_split(meta_TESTVAL, y2, test_size = 0.5, random_state = 123, stratify = y2)

print("meta Train :\n", meta_TRAIN["CLASS"].value_counts())
print("meta VAL :\n", meta_VAL["CLASS"].value_counts())
print("meta TEST :\n", meta_TEST["CLASS"].value_counts())
meta_TRAIN.head()

meta_TRAIN["SET"] = "TRAIN"
meta_VAL["SET"] = "VAL"
meta_TEST["SET"] = "TEST"
meta_dataset = pd.concat([meta_TRAIN, meta_VAL, meta_TEST], axis = 0)
meta_dataset["SET"].value_counts()


meta Train :
 VIRAL_PNEUMONIA    942
COVID              942
NORMAL             941
LUNG_OPACITY       941
Name: CLASS, dtype: int64
meta VAL :
 LUNG_OPACITY       202
VIRAL_PNEUMONIA    202
NORMAL             202
COVID              201
Name: CLASS, dtype: int64
meta TEST :
 NORMAL             202
COVID              202
LUNG_OPACITY       202
VIRAL_PNEUMONIA    201
Name: CLASS, dtype: int64


TRAIN    3766
VAL       807
TEST      807
Name: SET, dtype: int64

## Etape 3 : enregistrement des 3 datasets dans 3 sous-dossiers TRAIN, VAL, TEST du dossier PREPROCESSING_2

In [43]:
meta_dataset.head()

,CLASS,FILE NAME,FORMAT,SIZE,URL,SET
4080,VIRAL_PNEUMONIA,Viral Pneumonia-46,PNG,256*256,https://www.kaggle.com/paultimothymooney/chest...,TRAIN
1106,COVID,COVID-2461,PNG,256*256,https://bimcv.cipf.es/bimcv-projects/bimcv-cov...,TRAIN
4244,VIRAL_PNEUMONIA,Viral Pneumonia-210,PNG,256*256,https://www.kaggle.com/paultimothymooney/chest...,TRAIN
5306,VIRAL_PNEUMONIA,Viral Pneumonia-1272,PNG,256*256,https://www.kaggle.com/paultimothymooney/chest...,TRAIN
1058,COVID,COVID-1693,PNG,256*256,https://bimcv.cipf.es/bimcv-projects/bimcv-cov...,TRAIN


In [46]:
dossier_source = "..\\data\\Preprocessing_1"
dossiers_cibles_dict = {"TRAIN" : "..\\data\\Preprocessing_2\Train", "VAL" : "..\\data\\Preprocessing_2\Train", "TEST" : "..\\data\\Preprocessing_2\Train"}

#Fonction pour copier les fichiers dans le dossier cible
def copy_to_folder(filename, source_folder, target_folder):
    filename = filename + ".png"
    #chemin d'accès au fichier soruce
    src = os.path.join(source_folder, filename)
    #chemin d'accès du fichier copié
    dst = os.path.join(target_folder, filename)
    #copie
    shutil.copyfile(src, dst)

#Itération sur les noms de fichiers retenus par le TRAIN TEST SPLIT
for index, row in tqdm(meta_dataset.iterrows()):
    dossier_cible = dossiers_cibles_dict[row["SET"]]
    copy_to_folder(row["FILE NAME"], dossier_source, dossier_cible)

5380it [01:45, 50.92it/s]
